# XMM-LSS master catalogue

This notebook presents the merge of the various pristine catalogues to produce the HELP master catalogue on XMM-LSS.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
33f5ec7 (Wed Dec 6 16:56:17 2017 +0000) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [4]:
#candels = Table.read("{}/CANDELS.fits".format(TMP_DIR))           # 1.1
#cfht_wirds = Table.read("{}/CFHT-WIRDS.fits".format(TMP_DIR))     # 1.3
#cfhtls_wide = Table.read("{}/CFHTLS-WIDE.fits".format(TMP_DIR))   # 1.4a
#cfhtls_deep = Table.read("{}/CFHTLS-DEEP.fits".format(TMP_DIR))   # 1.4b
#We no longer use CFHTLenS as it is the same raw data set as CFHTLS-WIDE
# cfhtlens = Table.read("{}/CFHTLENS.fits".format(TMP_DIR))         # 1.5
#decals = Table.read("{}/DECaLS.fits".format(TMP_DIR))             # 1.6
#servs = Table.read("{}/SERVS.fits".format(TMP_DIR))               # 1.8
#swire = Table.read("{}/SWIRE.fits".format(TMP_DIR))               # 1.7
#hsc_wide = Table.read("{}/HSC-WIDE.fits".format(TMP_DIR))         # 1.9a
#hsc_deep = Table.read("{}/HSC-DEEP.fits".format(TMP_DIR))         # 1.9b
#hsc_udeep = Table.read("{}/HSC-UDEEP.fits".format(TMP_DIR))       # 1.9c
#ps1 = Table.read("{}/PS1.fits".format(TMP_DIR))                   # 1.10
#sxds = Table.read("{}/SXDS.fits".format(TMP_DIR))                 # 1.11
#sparcs = Table.read("{}/SpARCS.fits".format(TMP_DIR))             # 1.12
dxs = Table.read("{}/UKIDSS-DXS.fits".format(TMP_DIR))            # 1.13
uds = Table.read("{}/UKIDSS-UDS.fits".format(TMP_DIR))            # 1.14
#vipers = Table.read("{}/VIPERS.fits".format(TMP_DIR))             # 1.15
#vhs = Table.read("{}/VISTA-VHS.fits".format(TMP_DIR))             # 1.16
#video = Table.read("{}/VISTA-VIDEO.fits".format(TMP_DIR))         # 1.17
#viking = Table.read("{}/VISTA-VIKING.fits".format(TMP_DIR))       # 1.18

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones. We start with PanSTARRS because it coevrs the whole field.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### Start with DXS

In [5]:
master_catalogue = dxs
master_catalogue['dxs_ra'].name = 'ra'
master_catalogue['dxs_dec'].name = 'dec'

### Add UDS

In [6]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(uds['uds_ra'], uds['uds_dec'])
)

HELP Warning: There weren't any cross matches. The two surveys probably don't overlap.


In [7]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, uds, "uds_ra", "uds_dec", radius=0.8*u.arcsec)

### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [8]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col] = master_catalogue[col].astype(float)
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [9]:
#Since this is not the final merged catalogue. We rename column names to make them unique
master_catalogue['ra'].name = 'ukidss_ra'
master_catalogue['dec'].name = 'ukidss_dec'
master_catalogue['flag_merged'].name = 'ukidss_flag_merged'

In [10]:
master_catalogue[:10].show_in_notebook()

idx,dxs_id,ukidss_ra,ukidss_dec,m_ap_ukidss_j,merr_ap_ukidss_j,m_ukidss_j,merr_ukidss_j,m_ap_ukidss_k,merr_ap_ukidss_k,m_ukidss_k,merr_ukidss_k,dxs_stellarity,f_ap_ukidss_j,ferr_ap_ukidss_j,f_ukidss_j,ferr_ukidss_j,flag_ukidss_j,f_ap_ukidss_k,ferr_ap_ukidss_k,f_ukidss_k,ferr_ukidss_k,flag_ukidss_k,dxs_flag_cleaned,dxs_flag_gaia,ukidss_flag_merged,uds_id,m_ap_uds_j,merr_ap_uds_j,m_uds_j,merr_uds_j,m_ap_uds_h,merr_ap_uds_h,m_uds_h,merr_uds_h,m_ap_uds_k,merr_ap_uds_k,m_uds_k,merr_uds_k,uds_stellarity,f_ap_uds_j,ferr_ap_uds_j,f_uds_j,ferr_uds_j,flag_uds_j,f_ap_uds_h,ferr_ap_uds_h,f_uds_h,ferr_uds_h,flag_uds_h,f_ap_uds_k,ferr_ap_uds_k,f_uds_k,ferr_uds_k,flag_uds_k,uds_flag_cleaned,uds_flag_gaia
,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,446679621523,35.5523293864,-3.89660860231,11.3550081253,0.000288982642815,9.84205055237,0.000124145561131,nan,nan,nan,nan,0.899999976158,104230.921875,27.742401123,419932.625,48.0160560608,False,nan,nan,nan,nan,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
1,446679652426,35.5348871949,-3.71725661147,11.4726924896,0.000298160797684,11.3987607956,0.000252233905485,12.2191505432,0.000333435629727,12.4362030029,0.000336990662618,0.993865013123,93523.953125,25.6831951141,100114.164062,23.2581233978,False,47026.1757812,14.4420013428,38505.1289062,11.9512176514,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
2,446679622311,35.4618945331,-3.88980341943,11.8227844238,0.000358827644959,12.1339998245,0.000361646903912,12.485329628,0.000376348238206,13.0113334656,0.000440127478214,0.993865013123,67746.3828125,22.3896713257,50862.7578125,16.9418315887,False,36801.7109375,12.7565593719,22670.78125,9.19010829926,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
3,446679621401,35.5605843318,-3.7944250223,12.2684221268,0.000439715222456,12.2972803116,0.000388649466913,12.8169708252,0.000437340902863,12.7985486984,0.000394046044676,0.993865013123,44939.7890625,18.2002868652,43761.0507812,15.6646785736,False,27115.1210938,10.9221286774,27579.1171875,10.0092840195,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
4,446679664917,35.5402460454,-3.98118188052,12.5791101456,0.000503239629325,13.0774402618,0.000557364197448,12.872795105,0.000448075530585,13.2749996185,0.000487160374178,0.993865013123,33756.3789062,15.6461105347,21331.6132812,10.9506139755,False,25756.1992188,10.6293983459,17782.7949219,7.9789853096,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
5,446679621505,35.5515207051,-3.79330844275,13.0234146118,0.000622916268185,13.0428237915,0.000552586105186,nan,nan,nan,nan,0.899999976158,22419.9160156,12.862912178,22022.6894531,11.2084617615,False,nan,nan,nan,nan,False,False,2,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
6,446679652126,35.4290092673,-3.7446369154,13.073425293,0.000624368316494,13.2542324066,0.00060314568691,13.6870708466,0.000657663622405,13.7026987076,0.000604641973041,0.993865013123,21410.6425781,12.3125,18126.2050781,10.0694274902,False,12166.6621094,7.36971902847,11992.7939453,6.67873668671,False,False,3,False,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,False,0
7,446679621482,35.5537078675,-3.91012275129,13.0470628738,0.000630565860774,12.9837150574,0.000537223299034,13.7780056,0.00068298302358,13.7701282501,0.000624908541795,0.993865013123,21936.8730469,12.7403354645,23254.8652344,11.506529808,False,11189.1630859

## V - Adding unique identifier

In [11]:
master_catalogue.add_column(Column(data=(np.char.array(master_catalogue['dxs_id'].astype(str)) 
                                    +  np.char.array(master_catalogue['uds_id'].astype(str) )), 
                              name="ukidss_intid"))



In [12]:
id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

['dxs_id', 'uds_id', 'ukidss_intid']


## VII - Choosing between multiple values for the same filter



### VII. d UKIDSS DXS and UDS

There is no overlap between UDS and DXS so I simply merge the two columns.

In [13]:
#Band H is only in UDS so we can simply rename it
for col in master_catalogue.colnames:
    if 'uds_h' in col:
        master_catalogue[col].name = col.replace('uds_h', 'ukidss_h')
        
has_uds_k =     ~np.isnan(master_catalogue['f_uds_k'])
has_uds_j =     ~np.isnan(master_catalogue['f_uds_j'])
has_ukidss_k =  ~np.isnan(master_catalogue['f_ukidss_k'])
has_ukidss_j =  ~np.isnan(master_catalogue['f_ukidss_j'])

master_catalogue['f_ukidss_k'][has_uds_k] = master_catalogue['f_uds_k'][has_uds_k]
master_catalogue['ferr_ukidss_k'][has_uds_k] = master_catalogue['ferr_uds_k'][has_uds_k]
master_catalogue['m_ukidss_k'][has_uds_k] = master_catalogue['m_uds_k'][has_uds_k]
master_catalogue['merr_ukidss_k'][has_uds_k] = master_catalogue['merr_uds_k'][has_uds_k]
master_catalogue['flag_ukidss_k'][has_uds_k] = master_catalogue['flag_uds_k'][has_uds_k]

master_catalogue['f_ukidss_j'][has_uds_j] = master_catalogue['f_uds_j'][has_uds_j]
master_catalogue['ferr_ukidss_j'][has_uds_j] = master_catalogue['ferr_uds_j'][has_uds_j]
master_catalogue['m_ukidss_j'][has_uds_j] = master_catalogue['m_uds_j'][has_uds_j]
master_catalogue['merr_ukidss_j'][has_uds_j] = master_catalogue['merr_uds_j'][has_uds_j]
master_catalogue['flag_ukidss_j'][has_uds_j] = master_catalogue['flag_uds_j'][has_uds_j]

has_ap_uds_k =  ~np.isnan(master_catalogue['f_ap_uds_k'])
has_ap_uds_j =  ~np.isnan(master_catalogue['f_ap_uds_j'])
has_ap_ukidss_k =  ~np.isnan(master_catalogue['f_ap_ukidss_k'])
has_ap_ukidss_j =  ~np.isnan(master_catalogue['f_ap_ukidss_j'])

master_catalogue['f_ap_ukidss_k'][has_ap_uds_k] = master_catalogue['f_ap_uds_k'][has_ap_uds_k]
master_catalogue['ferr_ap_ukidss_k'][has_ap_uds_k] = master_catalogue['ferr_ap_uds_k'][has_ap_uds_k]
master_catalogue['m_ap_ukidss_k'][has_ap_uds_k] = master_catalogue['m_ap_uds_k'][has_ap_uds_k]
master_catalogue['merr_ap_ukidss_k'][has_ap_uds_k] = master_catalogue['merr_ap_uds_k'][has_ap_uds_k]

master_catalogue['f_ap_ukidss_j'][has_ap_uds_j] = master_catalogue['f_ap_uds_j'][has_ap_uds_j]
master_catalogue['ferr_ap_ukidss_j'][has_ap_uds_j] = master_catalogue['ferr_ap_uds_j'][has_ap_uds_j]
master_catalogue['m_ap_ukidss_j'][has_ap_uds_j] = master_catalogue['m_ap_uds_j'][has_ap_uds_j]
master_catalogue['merr_ap_ukidss_j'][has_ap_uds_j] = master_catalogue['merr_ap_uds_j'][has_ap_uds_j]

master_catalogue.remove_columns(['f_uds_j','ferr_uds_j','m_uds_j','merr_uds_j','flag_uds_j',
                               'f_uds_k','ferr_uds_k','m_uds_k','merr_uds_k','flag_uds_k',
                               'f_ap_uds_j','ferr_ap_uds_j','m_ap_uds_j','merr_ap_uds_j',
                               'f_ap_uds_k','ferr_ap_uds_k','m_ap_uds_k','merr_ap_uds_k'])


ukidss_origin = Table()
ukidss_origin.add_column(master_catalogue['ukidss_intid'])
origin = np.full(len(master_catalogue), '     ', dtype='<U5')
origin[has_uds_k] = "UDS"
origin[has_ukidss_k] = "DXS"
ukidss_origin.add_column(Column(data=origin, name= 'f_ukidss_k' ))
origin = np.full(len(master_catalogue), '     ', dtype='<U5')
origin[has_uds_j] = "UDS"
origin[has_ukidss_j] = "DXS"
ukidss_origin.add_column(Column(data=origin, name= 'f_ukidss_j' ))
origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
origin_ap[has_ap_uds_k] = "UDS"
origin_ap[has_ap_ukidss_k] = "DXS"
ukidss_origin.add_column(Column(data=origin_ap, name= 'f_ap_ukidss_k' ))
origin_ap = np.full(len(master_catalogue), '     ', dtype='<U5')
origin_ap[has_ap_uds_j] = "UDS"
origin_ap[has_ap_ukidss_j] = "DXS"
ukidss_origin.add_column(Column(data=origin_ap, name= 'f_ap_ukidss_j' ))

ukidss_origin.write("{}/xmm-lss_ukidss_fluxes_origins{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)

## IX - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogues. This can be used to easily get additional information from them.

For convenience, we also cross-match the master list with the SDSS catalogue and add the objID associated with each source, if any. **TODO: should we correct the astrometry with respect to Gaia positions?**

## XI - Saving the catalogue

In [15]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    
    
columns += ["stellarity", "stellarity_origin", "flag_cleaned", "flag_merged", "flag_gaia", "flag_optnir_obs", "flag_optnir_det", 
            "zspec", "zspec_qual", "zspec_association_flag", "ebv"]

In [16]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

Missing columns: {'uds_flag_cleaned', 'ukidss_intid', 'uds_stellarity', 'ukidss_dec', 'dxs_stellarity', 'dxs_flag_gaia', 'dxs_id', 'uds_flag_gaia', 'uds_id', 'dxs_flag_cleaned', 'ukidss_flag_merged', 'ukidss_ra'}


In [18]:
master_catalogue.write("{}/ukidss_merged_catalogue_xmm-lss.fits".format(TMP_DIR), overwrite = True)